In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

## Langsmith Tracking and tracing

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [5]:
from langchain.document_loaders import PyMuPDFLoader  # or UnstructuredPDFLoader, PDFPlumberLoader
import os

In [6]:
# Directory where your PDFs are stored
pdf_dir = "data"

# Get list of PDF files
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]

# Load all documents
all_docs = []

for pdf_file in pdf_files:
    file_path = os.path.join(pdf_dir, pdf_file)
    loader = PyMuPDFLoader(file_path)
    docs = loader.load()
    all_docs.extend(docs)

# Now all_docs is a list of Document objects ready for processing
print(f"Loaded {len(all_docs)} documents from {len(pdf_files)} PDFs.")



Loaded 1096 documents from 75 PDFs.


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(all_docs)

print(f"Total chunks: {len(split_docs)}")


Total chunks: 1377


In [8]:
from langchain.embeddings import OpenAIEmbeddings

# Requires OPENAI_API_KEY set in your environment
embedding_model = OpenAIEmbeddings()


C:\Users\Chandan\AppData\Local\Temp\ipykernel_29492\1856709743.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()


In [9]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(split_docs, embedding_model)

# Optional: Save to disk
vectorstore.save_local("my_faiss_index")


In [12]:
# ✅ Step 4: Ask questions using RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(temperature=0)

qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 🧠 Ask a question!
query = "What is the main topic discussed in the PDFs?"
response = qa.run(query)

print("Answer:", response)


C:\Users\Chandan\AppData\Local\Temp\ipykernel_29492\3832447095.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa.run(query)


Answer: The main topics discussed in the PDFs are Dataframes, Recap of Concepts Learnt, What is Big Data - 4Vs, Introduction to Hadoop Core and its Ecosystem, and Distributed Framework.
